In [1]:
import numpy as np

In [2]:
from drmdp import feats, task
from drmdp.envs import gem

In [3]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="pos-enf", max_episode_steps=2500)
env

<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

## Control: Current Impl - neg-enf

In [4]:
MAX_STEPS = 2500

In [5]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="esp-neg", max_episode_steps=MAX_STEPS)

In [6]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>,
 <drmdp.core.EnvMonitor at 0x3219e5f10>)

In [7]:
rew_delay = task.reward_delay_distribution(None)

In [8]:
rew_delay

In [9]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [10]:
feats_spec = {"name": "scale", "args": None}

In [11]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [12]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [13]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 1607, Mean returns: -2147483845.6285796;


2025-04-16 15:25:00,538 INFO root: Episode 100 mean returns: -1438814345.271099


Episode: 100; Steps: 811, Mean returns: -1445830873.8849823;


2025-04-16 15:25:16,112 INFO root: Episode 200 mean returns: -1342177605.192196


Episode: 200; Steps: 2500, Mean returns: -1335500107.3969889;


...

2025-04-16 15:27:57,675 INFO root: Episode 300 mean returns: -1374389837.742562


Episode: 300; Steps: 2500, Mean returns: -1369823760.3916676;


2025-04-16 15:28:12,572 INFO root: Episode 400 mean returns: -1363652418.418463


Episode: 400; Steps: 2500, Mean returns: -1360251790.005094;


2025-04-16 15:28:26,643 INFO root: Episode 500 mean returns: -1391569705.698833


In [14]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 2500, Mean returns: -362.10844229530426;


2025-04-16 15:28:34,453 INFO root: Episode 100 mean returns: -1954210282.073675


Episode: 100; Steps: 277, Mean returns: -1956123880.049385;


2025-04-16 15:28:40,884 INFO root: Episode 200 mean returns: -2050847038.637289


Episode: 200; Steps: 786, Mean returns: -2051327818.7286522;


...

2025-04-16 15:29:55,855 INFO root: Episode 300 mean returns: -2126008951.057826


Episode: 300; Steps: 204, Mean returns: -2126080295.711313;


2025-04-16 15:30:02,268 INFO root: Episode 400 mean returns: -2131377655.369529


Episode: 400; Steps: 1218, Mean returns: -2131417820.3222842;


2025-04-16 15:30:09,480 INFO root: Episode 500 mean returns: -2126008947.807034


## Control: Current Impl - pos-enf

In [15]:
MAX_STEPS = 2500

In [16]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="pos-enf", max_episode_steps=MAX_STEPS)

In [17]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>,
 <drmdp.core.EnvMonitor at 0x329dfa390>)

In [18]:
rew_delay = task.reward_delay_distribution(None)

In [19]:
rew_delay

In [20]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [21]:
feats_spec = {"name": "scale", "args": None}

In [22]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [23]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [24]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 1382, Mean returns: 2594.8997145829303;


2025-04-16 15:30:24,451 INFO root: Episode 100 mean returns: 2841.897427


Episode: 100; Steps: 2500, Mean returns: 2860.1451054542035;


2025-04-16 15:30:38,789 INFO root: Episode 200 mean returns: 2754.337944


Episode: 200; Steps: 1751, Mean returns: 2756.3124065910606;


...

2025-04-16 15:33:30,262 INFO root: Episode 300 mean returns: 2761.129893


Episode: 300; Steps: 2150, Mean returns: 2764.852551817877;


2025-04-16 15:33:45,665 INFO root: Episode 400 mean returns: 2795.537443


Episode: 400; Steps: 2500, Mean returns: 2799.94953876378;


2025-04-16 15:34:01,605 INFO root: Episode 500 mean returns: 2841.224987


In [25]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 68, Mean returns: 122.88654753349549;


2025-04-16 15:34:02,409 INFO root: Episode 100 mean returns: 122.952850


Episode: 100; Steps: 72, Mean returns: 123.04194932178902;


2025-04-16 15:34:03,184 INFO root: Episode 200 mean returns: 122.178938


Episode: 200; Steps: 70, Mean returns: 122.19253672172911;


...

2025-04-16 15:34:17,688 INFO root: Episode 300 mean returns: 160.294370


Episode: 300; Steps: 93, Mean returns: 160.28888874975624;


2025-04-16 15:34:18,674 INFO root: Episode 400 mean returns: 160.445555


Episode: 400; Steps: 93, Mean returns: 160.48549552470396;


2025-04-16 15:34:21,992 INFO root: Episode 500 mean returns: 238.106428


## Control: Current Impl - default

In [26]:
MAX_STEPS = 2500

In [27]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="default", max_episode_steps=MAX_STEPS)

In [28]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>,
 <drmdp.core.EnvMonitor at 0x329e266d0>)

In [29]:
rew_delay = task.reward_delay_distribution(None)

In [30]:
rew_delay

In [31]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [32]:
feats_spec = {"name": "scale", "args": None}

In [33]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<DiscretiseActionWrapper<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>>

In [34]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [35]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 752, Mean returns: -70.10032948100827;


2025-04-16 15:34:37,265 INFO root: Episode 100 mean returns: -302.079893


Episode: 100; Steps: 2500, Mean returns: -303.4738887143251;


2025-04-16 15:34:52,879 INFO root: Episode 200 mean returns: -314.846280


Episode: 200; Steps: 838, Mean returns: -314.18098650548654;


...

2025-04-16 15:37:45,399 INFO root: Episode 300 mean returns: -316.705146


Episode: 300; Steps: 1905, Mean returns: -317.1428331031952;


2025-04-16 15:38:00,459 INFO root: Episode 400 mean returns: -311.637846


Episode: 400; Steps: 1964, Mean returns: -311.44293378156146;


2025-04-16 15:38:15,292 INFO root: Episode 500 mean returns: -310.195893


In [36]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )

    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )

    for episode, snapshot in enumerate(results):
        if episode % (num_episodes // 5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 2500, Mean returns: -325.5354952889181;


2025-04-16 15:38:24,828 INFO root: Episode 100 mean returns: -178.461430


Episode: 100; Steps: 85, Mean returns: -176.8469094616986;


2025-04-16 15:38:28,925 INFO root: Episode 200 mean returns: -129.388932


Episode: 200; Steps: 90, Mean returns: -128.82244223307262;


...

2025-04-16 15:39:08,499 INFO root: Episode 300 mean returns: -96.963796


Episode: 300; Steps: 83, Mean returns: -96.72234310058474;


2025-04-16 15:39:09,890 INFO root: Episode 400 mean returns: -78.649784


Episode: 400; Steps: 100, Mean returns: -78.48573128940544;


2025-04-16 15:39:11,335 INFO root: Episode 500 mean returns: -67.512205


Regardless of the reward function, a random policy outperforms epsilon greedy on this problem.
The reason why must be investigated.